# DS 6014 Final Project: Bayesian Analysis of Refugee Asylum Data

- Youssef Abubaker; yaa2vb
- Joseph Cho; jc4jx
- Andrew J. Graves; ajg3eh
- Gabe Yohe; gjy7kb

In [ ]:
# Import modules
import math
import numpy as np
import pandas as pd
import scipy as sp
import pymc3 as pm
import matplotlib.pyplot as plt
import seaborn as sns
import arviz as az
import plotly.express as px

# Format plots: Uncomment the following lines for prettier plots
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf', dpi=300)

## Helper functions

In [ ]:
# Function for computing random indices
def get_rand_indices(df, col):
    # Get unique values
    unique = df[col].unique()
    n = len(unique)
    
    # Get indexing mechanism
    lookup = dict(zip(unique, range(n)))
    out = df[col].replace(lookup).values
    return out, n, lookup

# Fucntion for wrangling model and map data
def map_params(rand_name, is_origin=True, is_intercept=True):
    
    if is_origin:
        col_str = 'origin'
        ID = country_origin
        lookup = country_origin_lookup
    else:
        col_str = 'asylum'
        ID = country_asylum
        lookup = country_asylum_lookup
    
    # Get summary statistic from trace
    if is_intercept:
        stat = sp.special.expit(trace[rand_name].mean(axis=0))
    else:
        stat = trace[rand_name].mean(axis=0)
    # Stack stats with identifier
    params = pd.DataFrame(np.vstack((np.unique(ID), stat)).T, columns=['lookup', 'param'])
    # Get the country names
    country_names = pd.concat((pd.Series(lookup).reset_index(), params), axis=1)
    # Merge the appropriate dataframes
    map_df = df.merge(country_names, 
                       left_on=f'Country of {col_str}', right_on='index').merge(
                       raw_df[[f'Country of {col_str}', f'Country of {col_str} (ISO)']].drop_duplicates(), 
                                on=f'Country of {col_str}')
    # Build final dataframe for plotting
    plot_map_df = map_df[[f'Country of {col_str}', f'Country of {col_str} (ISO)', 'param']].drop_duplicates()
    return plot_map_df

## Clean the data

In [ ]:
# Read in the asylum data
raw_df = pd.read_csv('data/asylum-decisions.csv', index_col=False)

# Rename the columns
raw_df.rename({'Recognized decisions': 'Cases_persons', 
           'Other decisions': 'Recognized decisions', 
           'Rejected decisions': 'Complementary_Protection',
           'dec_closed': 'Rejected decisions',
           'dec_total': 'Otherwise closed'}, axis=1, inplace=True)

# Drop columns that contain NA
df = raw_df.dropna(how='any', axis=1)

# Get the outcome sums
df = df.groupby(['Year', 'Country of origin', 'Country of asylum']).sum().reset_index()

# Compute rejection rate
df['prop_rejected'] = (df['Rejected decisions']) / (
    df['Recognized decisions'] + df['Rejected decisions'] + 
    df['Complementary_Protection'] + df['Otherwise closed'])

# Smooth 0 and 1 probabilities 
outcome = np.array(df.prop_rejected.replace({1: 1 - 1e-5, 0: 1e-5}))

# Get weights by counting each decision row-wise
weights = np.array(df[['Recognized decisions', 'Complementary_Protection', 
    'Rejected decisions', 'Otherwise closed']].sum(axis=1))
df['weights'] = weights

# Scale year
year = (df.Year - np.mean(df.Year)) / np.std(df.Year)

# Get indexers for random intercepts and random slopes
country_origin, n_country_origin, country_origin_lookup = get_rand_indices(df, 'Country of origin')
country_asylum, n_country_asylum, country_asylum_lookup = get_rand_indices(df, 'Country of asylum')

## Fit the model

In [ ]:
def fit_model(origin, asylum, n_origin, n_asylum):
    
    with pm.Model() as model:

        # Initialize prior lists
        prior_mu = []
        prior_sd = []
        rand_efs = []
        
        # Specify random effects labels
        labs = ['Random Intercept Origin', 'Random Intercept Asylum',
                'Random Slope Origin', 'Random Slope Asylum']

        # Priors for random intercepts and slopes
        for i in range(4):
            if not i % 2:
                shape = n_origin
            else:
                shape = n_asylum
            prior_mu.append(pm.Normal(f'Prior mean {i}', mu=0, sd=sp.special.logit(outcome).std()*10))
            prior_sd.append(pm.HalfCauchy(f'Prior sd {i}', 10))
            rand_efs.append(pm.Normal(f'{labs[i]}', 
                                      mu=prior_mu[i], sd=prior_sd[i], shape=shape))

        # Fit random slopes and intercepts for country of origin and asylum
        mu = rand_efs[0][origin] + rand_efs[1][asylum] + rand_efs[2][origin]*year + rand_efs[3][asylum]*year
        
        # Model error
        eps = pm.HalfCauchy('Model Error', 10)

        # Fit a Gaussian likelihood on a weighted logit of the rejection rates
        Y = pm.Potential('Weighted Logit Rejection Rates', 
                         weights*pm.Normal.dist(mu=mu, sd=eps).logp(sp.special.logit(outcome)))

        # Run ADVI
        approx = pm.fit(15000, method='advi', random_seed=100)

    return model, approx

# Fit random intercpets and slopes for each country of origin / asylum
mod, mf_approx = fit_model(country_origin, country_asylum, n_country_origin, n_country_asylum)

# Save the graphical model
g = pm.model_to_graphviz(mod)
g.render("graphname", format='pdf', cleanup=True);

## Check ELBO convergence

In [ ]:
plt.plot(np.arange(mf_approx.hist.shape[0]), -mf_approx.hist)
plt.xlabel('Number of iterations')
plt.ylabel('ELBO')
plt.title('Optimization process for variational inference')
plt.show()

## Sample the posterior

In [ ]:
with mod:
    # Sample the posterior
    trace = mf_approx.sample(5000)
    az.plot_trace(trace, compact=True)

In [ ]:
# Extract parameters in probability space
stat = sp.special.expit((trace['Random Intercept Asylum'])).mean(axis=0)
params = pd.DataFrame(np.vstack((np.unique(country_asylum), stat)).T, 
                      columns=['lookup', 'param'])
country_names = pd.concat((pd.Series(country_asylum_lookup).reset_index(), params), 
                          axis=1)
params = country_names[['index', 'param']].sort_values('index')

# Get weighted means
wm = lambda x: np.average(x, weights=df.loc[x.index, 'weights'])
weighted_means = pd.DataFrame(df.groupby('Country of asylum').prop_rejected.agg(wm)).reset_index()
comps = params.merge(weighted_means, left_on='index', right_on='Country of asylum')

# Check correlation between estimated probabilities and condtional weighted means
comps.corr().iloc[0, 1]

## Plot the results

In [ ]:
# Plot the trajectory for all countries
all_years = df.groupby('Year').prop_rejected.mean().reset_index()
sns.regplot(x='Year', y='prop_rejected', data=all_years)
plt.title('Rejection rates have globally decreased over the last 20 years')
plt.xticks(np.arange(2000, 2019, 2))
plt.ylabel('Probability of Rejection')
plt.show()

In [ ]:
# Plot the trajectory for Denmark
all_years = df.query("`Country of asylum` == 'Denmark' & Year > 2009").groupby('Year').prop_rejected.mean().reset_index()
sns.regplot(x='Year', y='prop_rejected', data=all_years)
plt.title('Denmark rejection rates have slowly crept upwards')
plt.xticks(np.arange(2010, 2019, 2))
plt.ylabel('Probability of Rejection')
plt.show()

In [ ]:
# PLot random intercepts for origin
fig = px.choropleth(map_params('Random Intercept Origin'), 
                    locations='Country of origin (ISO)',
                    color='param',
                    labels={'param': 'Intercept'},
                    color_continuous_scale=px.colors.sequential.Plasma,
                    title = 'Mapping probability of rejection across countries of origin')
fig.show()

In [ ]:
# PLot random intercepts for asylum
fig = px.choropleth(map_params('Random Intercept Asylum', is_origin=False), 
                    locations='Country of asylum (ISO)',
                    color='param',
                    labels={'param': 'Intercept'},
                    color_continuous_scale=px.colors.sequential.Plasma,
                    title = 'Mapping probability of rejection across countries of asylum')
fig.show()

In [ ]:
# PLot random slopes for origin
fig = px.choropleth(map_params('Random Slope Origin', is_intercept=False), 
                    locations='Country of origin (ISO)',
                    color='param',
                    labels={'param': 'Slope'},
                    color_continuous_scale=px.colors.sequential.Plasma,
                    title = 'Mapping latent rate of change for rejection rates across countries of origin')
fig.show()

In [ ]:
# PLot random slopes for asylum
fig = px.choropleth(map_params('Random Slope Asylum', is_origin=False, is_intercept=False), 
                    locations='Country of asylum (ISO)',
                    color='param',
                    labels={'param': 'Slope'},
                    color_continuous_scale=px.colors.sequential.Plasma,
                    title = 'Mapping latent rate of change for rejection rates across countries of asylum')
fig.show()